In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_7_1000_17300_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()

VGG_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.1, inplace=True)
    (14): Conv2d(

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [6]:
def process_num(x):
    return x*random.uniform(0.96, 1.04)

In [54]:
dataset_names = ['Grasp_dataset_banana', 'Grasp_dataset_chicken', 'Grasp_dataset_tomato', 'Grasp_dataset_ibuprofen']

header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_5_test', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break



p1_x_arr = np.array(p1_x_arr) - np.array(p1_x_arr)
p2_x_arr = np.array(p2_x_arr) - np.array(p1_x_arr)
p3_x_arr = np.array(p3_x_arr) - np.array(p1_x_arr)
p4_x_arr = np.array(p4_x_arr) - np.array(p1_x_arr)
p5_x_arr = np.array(p5_x_arr) - np.array(p1_x_arr)
p6_x_arr = np.array(p6_x_arr) - np.array(p1_x_arr)
p7_x_arr = np.array(p7_x_arr) - np.array(p1_x_arr)
p8_x_arr = np.array(p8_x_arr) - np.array(p1_x_arr)
p9_x_arr = np.array(p9_x_arr) - np.array(p1_x_arr)
p10_x_arr = np.array(p10_x_arr) - np.array(p1_x_arr)
p11_x_arr = np.array(p11_x_arr) - np.array(p1_x_arr)
p12_x_arr = np.array(p12_x_arr) - np.array(p1_x_arr)
p13_x_arr = np.array(p13_x_arr) - np.array(p1_x_arr)
p14_x_arr = np.array(p14_x_arr) - np.array(p1_x_arr)
p15_x_arr = np.array(p15_x_arr) - np.array(p1_x_arr)
p16_x_arr = np.array(p16_x_arr) - np.array(p1_x_arr)

p1_y_arr = np.array(p1_y_arr) - np.array(p1_y_arr)
p2_y_arr = np.array(p2_y_arr) - np.array(p1_y_arr)
p3_y_arr = np.array(p3_y_arr) - np.array(p1_y_arr)
p4_y_arr = np.array(p4_y_arr) - np.array(p1_y_arr)
p5_y_arr = np.array(p5_y_arr) - np.array(p1_y_arr)
p6_y_arr = np.array(p6_y_arr) - np.array(p1_y_arr)
p7_y_arr = np.array(p7_y_arr) - np.array(p1_y_arr)
p8_y_arr = np.array(p8_y_arr) - np.array(p1_y_arr)
p9_y_arr = np.array(p9_y_arr) - np.array(p1_y_arr)
p10_y_arr = np.array(p10_y_arr) - np.array(p1_y_arr)
p11_y_arr = np.array(p11_y_arr) - np.array(p1_y_arr)
p12_y_arr = np.array(p12_y_arr) - np.array(p1_y_arr)
p13_y_arr = np.array(p13_y_arr) - np.array(p1_y_arr)
p14_y_arr = np.array(p14_y_arr) - np.array(p1_y_arr)
p15_y_arr = np.array(p15_y_arr) - np.array(p1_y_arr)
p16_y_arr = np.array(p16_y_arr) - np.array(p1_y_arr)


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(force_reading_2_arr) - np.array(weight_reading_2_arr) 

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)

# num_cols = grasp_dataset.select_dtypes(include=['float']).columns
# df[num_cols] = grasp_dataset[num_cols].applymap(process_num)

# combined_df_set = grasp_dataset
# combined_df_set = pd.concat([combined_df_set, grasp_dataset], axis=0)


In [55]:
pd.options.display.max_columns = None
print(grasp_dataset.head(3))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0             134.1             112.9                59.5                97.9   
1             136.2             108.6                74.5                45.6   
2             126.9             118.3                64.4                63.2   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0            402.1            401.9   0.0   0.0  0.482460  0.153350  0.800311   
1            398.2            396.6   0.0   0.0  0.482028  0.141272  0.800172   
2            396.8            395.5   0.0   0.0  0.482992  0.142759  0.797908   

       p3_y      p4_x      p4_y      p5_x      p5_y     p6_x      p6_y  \
0  0.361048  0.796652  0.688427  0.476602  0.934279  0.17459  0.694402   
1  0.366549  0.795772  0.693964  0.473183  0.922507  0.17513  0.691282   
2  0.371525  0.794061  0.703353  0.471885  0.923529  0.17889  0.691149   

       p7_x      p7_y      p8_x      p8_y      p9_x  

In [56]:
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,134.1,112.9,59.5,97.9,402.1,401.9,0.0,0.0,0.482460,0.153350,0.800311,0.361048,0.796652,0.688427,0.476602,0.934279,0.174590,0.694402,0.467484,0.271226,0.455653,0.402846,0.452430,0.537536,0.458628,0.665563,0.465876,0.795476,0.497157,0.793216,0.513429,0.660194,0.520939,0.532275,0.514096,0.412721,0.501296,0.280402,banana
1,136.2,108.6,74.5,45.6,398.2,396.6,0.0,0.0,0.482028,0.141272,0.800172,0.366549,0.795772,0.693964,0.473183,0.922507,0.175130,0.691282,0.467680,0.265698,0.456829,0.398410,0.453426,0.533317,0.458943,0.661141,0.465047,0.789868,0.494889,0.788492,0.512347,0.656651,0.520281,0.528781,0.513294,0.408554,0.500638,0.274579,banana
2,126.9,118.3,64.4,63.2,396.8,395.5,0.0,0.0,0.482992,0.142759,0.797908,0.371525,0.794061,0.703353,0.471885,0.923529,0.178890,0.691149,0.467645,0.265147,0.456780,0.398302,0.453783,0.533722,0.459376,0.661249,0.464855,0.789792,0.493708,0.790707,0.510889,0.659347,0.518819,0.530992,0.511806,0.409284,0.499865,0.274812,banana
3,137.5,109.1,45.8,98.9,395.5,394.9,0.0,0.0,0.482804,0.151381,0.802029,0.350721,0.795673,0.688608,0.477061,0.930995,0.175447,0.694894,0.468965,0.271587,0.456845,0.403500,0.453185,0.538248,0.458932,0.666348,0.466652,0.796128,0.497113,0.792663,0.512898,0.659821,0.520899,0.531735,0.514785,0.411678,0.502335,0.278845,banana
4,137.3,109.1,51.0,99.0,389.0,388.5,0.0,0.0,0.483792,0.155989,0.800263,0.362284,0.794932,0.690051,0.477458,0.932092,0.175924,0.693846,0.468474,0.272432,0.456380,0.403280,0.452636,0.537839,0.458372,0.666414,0.466091,0.797012,0.496719,0.794167,0.511902,0.661119,0.519666,0.532657,0.513568,0.412224,0.501665,0.279966,banana
5,113.4,113.3,62.9,105.0,364.6,363.7,0.0,0.0,0.481428,0.163162,0.796497,0.346878,0.790081,0.692015,0.475290,0.924940,0.176793,0.698282,0.468547,0.269056,0.457615,0.399832,0.454727,0.535581,0.459883,0.664491,0.465971,0.793856,0.493770,0.793018,0.508533,0.660741,0.515756,0.531158,0.510216,0.409623,0.499091,0.278020,banana
6,121.0,105.1,44.5,96.3,401.7,400.9,0.0,0.0,0.484785,0.142892,0.802124,0.354782,0.796724,0.689236,0.475221,0.929872,0.175911,0.694725,0.467871,0.261491,0.455575,0.395172,0.452466,0.531162,0.458309,0.659900,0.465110,0.790674,0.494979,0.791084,0.511293,0.657611,0.519264,0.527496,0.512718,0.404915,0.501070,0.270424,banana
7,125.7,100.9,44.2,89.0,382.5,382.6,0.0,0.0,0.479090,0.148775,0.799499,0.342809,0.793299,0.689559,0.474615,0.925650,0.178382,0.697791,0.467747,0.265420,0.458367,0.398489,0.455686,0.534486,0.460787,0.662764,0.466360,0.791549,0.495076,0.789455,0.511441,0.657091,0.519076,0.528069,0.512569,0.406636,0.499560,0.272668,banana
8,118.1,107.8,66.9,62.0,383.1,382.9,0.0,0.0,0.484468,0.134546,0.802093,0.366323,0.792962,0.700503,0.468744,0.904905,0.177291,0.694285,0.468878,0.256978,0.457869,0.390733,0.454357,0.527271,0.459062,0.655514,0.463533,0.781986,0.491381,0.780655,0.510656,0.650766,0.520033,0.523321,0.514019,0.402672,0.502092,0.267071,banana
9,112.9,112.1,41.8,93.8,410.4,408.8,0.0,0.0,0.483630,0.140273,0.802054,0.350702,0.795704,0.689020,0.474706,0.927874,0.177257,0.695880,0.468152,0.260771,0.456589,0.394851,0.453568,0.531019,0.459179,0.659573,0.465473,0.789556,0.495140,0.788986,0.511966,0.655906,0.520123,0.526328,0.513574,0.404239,0.501371,0.269374,banana


In [ ]:
# grasp_dataset.to_csv('/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Grasp_dataset_3/' + 'validation_df.csv', index=False)

In [57]:
grasp_data = grasp_dataset.drop(columns=['label', 'weight_reading_1', 'weight_reading_2'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
# grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
# grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   pressure_reading_1  pressure_reading_2  force_reading_1  force_reading_2  \
0               0.595               0.979           0.4021           0.4019   
1               0.745               0.456           0.3982           0.3966   

   p1_x  p1_y      p2_x      p2_y      p3_x      p3_y      p4_x      p4_y  \
0   0.0   0.0  0.482460  0.153350  0.800311  0.361048  0.796652  0.688427   
1   0.0   0.0  0.482028  0.141272  0.800172  0.366549  0.795772  0.693964   

       p5_x      p5_y     p6_x      p6_y      p7_x      p7_y      p8_x  \
0  0.476602  0.934279  0.17459  0.694402  0.467484  0.271226  0.455653   
1  0.473183  0.922507  0.17513  0.691282  0.467680  0.265698  0.456829   

       p8_y      p9_x      p9_y     p10_x     p10_y     p11_x     p11_y  \
0  0.402846  0.452430  0.537536  0.458628  0.665563  0.465876  0.795476   
1  0.398410  0.453426  0.533317  0.458943  0.661141  0.465047  0.789868   

      p12_x     p12_y     p13_x     p13_y     p14_x     p14_y     p15_x  \
0  0.4

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=100, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# num_folds = 10
# cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
# cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

# print('Cross-validation results:', cv_results)
# print('Average accuracy:', cv_results.mean())

clf.fit(data_train, label_train)
label_pred = clf.predict(data_test)
cm = confusion_matrix(label_test, label_pred)

# short_dataset_names = ['chicken', 'tomato']


# cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

# print(cm_df)
# print(cm)

accuracy = clf.score(data_test, label_test)
print("Accuracy:", accuracy)

Accuracy: 0.925


In [58]:
label_pred = clf.predict(grasp_data)
print(label_pred)
accuracy = clf.score(grasp_data, grasp_label)
print("Accuracy:", accuracy)

['chicken' 'chicken' 'chicken' 'banana' 'banana' 'chicken' 'banana'
 'chicken' 'banana' 'banana' 'chicken' 'chicken' 'chicken' 'chicken'
 'chicken' 'chicken' 'chicken' 'chicken' 'chicken' 'chicken' 'tomato'
 'tomato' 'tomato' 'tomato' 'tomato' 'tomato' 'tomato' 'tomato' 'tomato'
 'tomato' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen'
 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen' 'ibuprofen']
Accuracy: 0.875


In [ ]:
Chicken, egg, tomato